# Generate topics using abstracts and keywords extracted from papers


Once keywords are extracted, this is an example on how to extract targeted topics (from these keywords) and associated different abstracts to the closest topics. In this example, I only focus in keywords of 3-grams 


In [1]:
import os 
import json
from collections import Counter


NEURIPS_ANALYSIS_DATA_PATH = '/var/data/neurips_analysis'

with open(os.path.join(NEURIPS_ANALYSIS_DATA_PATH, 'year_keywords_counter_n3.json'), 'r') as f:
    keywords_n3 = json.load(f)

print(f'keywords loaded -- 3-grams: {sum([len(v) for k, v in keywords_n3.items()])}')


all_n3_keywords = Counter()
for year, values in keywords_n3.items():
    all_n3_keywords.update([v[0] for v in values.items()])
selected_n3_keywords = [x[0] for x in all_n3_keywords.items() if x[1] > 1]
len(selected_n3_keywords)



keywords loaded -- 3-grams: 120341


2116

In [2]:
from collections import defaultdict

from tqdm.notebook import tqdm 

from papeles.paper.neurips import get_key
from papeles.utils import text as text_utils
from papeles.utils.topics import Topics

In [3]:
metadata_path = os.path.join(NEURIPS_ANALYSIS_DATA_PATH, 'files_metadata/')

metadata = {}
for filename in tqdm(os.listdir(metadata_path), 'loading metadata'):
    with open(os.path.join(metadata_path, filename), 'r') as f:  # open in readonly mode
        for line in f.readlines():
            data = json.loads(line)
            metadata[get_key(data['pdf_name'])] = data


## Topics extraction and topics per year


Topics are generated using the whole corpus (all abstracts from all years). Then, all topics are counter per year to generate a final analysis of which topics are the most common per year.  

In [4]:
topics = Topics([x['abstract'] for x in metadata.values()], selected_n3_keywords, random_state=42)
topics.topics

{'topic_0': ['gradient_langevin_dynamics',
  'stochastic_gradient_langevin',
  'langevin_monte_carlo',
  'learning_large_scale',
  'bayesian_learning_large',
  'machine_learning_tasks',
  'gradient_descent_sgd',
  'stochastic_gradient_descent',
  'low_dimensional_structures',
  'high_dimensional_datasets'],
 'topic_1': ['high_dimensional_datasets',
  'low_dimensional_structures',
  'paper_concerns_problem',
  'problem_learning_control',
  'superposition-structured_dirty_statistical',
  'contextual_bandits_learner',
  'concerns_problem_learning',
  'low-rank_tensor_decomposition',
  'dynamic_assortment_selection',
  'posteriori_map_assignment'],
 'topic_2': ['loss_functions_deep',
  'functions_deep_neural',
  'optimized_stochastic_gradient',
  'modern_deep_networks',
  'stochastic_gradient_descent',
  'gradient_descent_sgd',
  'deep_neural_networks',
  'high_dimensional_datasets',
  'low_dimensional_structures',
  'iterative_algorithm_based'],
 'topic_3': ['address_problem_estimating',


In [5]:
import json 


with open('/var/data/neurips_analysis/3grams_topics.json', 'w') as f:
    json.dump(topics.topics, f)


In [6]:
topics_per_year = {}
for _, data in metadata.items():
    year = data['year']
    if year not in topics_per_year:
        topics_per_year[year] = Counter()
    
    topic_prediction = topics.predict_topics(data['abstract'])
    if sum(topic_prediction.values()) > 0:
        top_prediction = [x[0] for x in sorted(topic_prediction.items(), key=lambda x: x[1], reverse=True) if x[1] > 0][:5]
        topics_per_year[year].update(top_prediction)

topics_per_year

{2019: Counter({'topic_28': 1,
          'topic_10': 33,
          'topic_31': 19,
          'topic_41': 12,
          'topic_16': 5,
          'topic_54': 8,
          'topic_7': 11,
          'topic_38': 4,
          'topic_25': 11,
          'topic_2': 101,
          'topic_19': 88,
          'topic_34': 63,
          'topic_52': 89,
          'topic_5': 5,
          'topic_17': 11,
          'topic_12': 8,
          'topic_30': 8,
          'topic_24': 7,
          'topic_37': 30,
          'topic_48': 39,
          'topic_45': 27,
          'topic_11': 20,
          'topic_44': 9,
          'topic_35': 45,
          'topic_22': 40,
          'topic_0': 50,
          'topic_6': 16,
          'topic_18': 19,
          'topic_20': 39,
          'topic_14': 23,
          'topic_8': 27,
          'topic_15': 25,
          'topic_26': 22,
          'topic_46': 28,
          'topic_21': 19,
          'topic_55': 23,
          'topic_27': 6,
          'topic_47': 5,
          'topic_23': 

In [7]:
for year in range(2009, 2020):
    print(f'===============\n{year}')
    topics = topics_per_year[year]
    print(sorted(topics.items(), key=lambda x: x[1], reverse=True)[:5])
    

2009
[('topic_9', 9), ('topic_45', 7), ('topic_10', 6), ('topic_27', 6), ('topic_11', 5)]
2010
[('topic_9', 7), ('topic_15', 6), ('topic_35', 6), ('topic_54', 6), ('topic_26', 5)]
2011
[('topic_10', 10), ('topic_35', 7), ('topic_22', 7), ('topic_25', 6), ('topic_45', 6)]
2012
[('topic_23', 8), ('topic_9', 7), ('topic_8', 6), ('topic_15', 6), ('topic_26', 6)]
2013
[('topic_8', 14), ('topic_18', 12), ('topic_15', 12), ('topic_46', 11), ('topic_26', 11)]
2014
[('topic_10', 15), ('topic_35', 14), ('topic_18', 13), ('topic_52', 13), ('topic_19', 11)]
2015
[('topic_2', 20), ('topic_52', 19), ('topic_35', 17), ('topic_10', 16), ('topic_0', 15)]
2016
[('topic_2', 40), ('topic_52', 36), ('topic_19', 31), ('topic_34', 24), ('topic_10', 22)]
2017
[('topic_2', 49), ('topic_52', 45), ('topic_19', 40), ('topic_22', 28), ('topic_0', 28)]
2018
[('topic_2', 78), ('topic_52', 71), ('topic_19', 70), ('topic_34', 47), ('topic_0', 40)]
2019
[('topic_2', 101), ('topic_52', 89), ('topic_19', 88), ('topic_34'